In [13]:
#r "nuget: Microsoft.SemanticKernel, *"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.18.1-alpha"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Experimental.Agents;
using System.IO;
using System.Linq;

class EnvLoader
{
    public static void Load(string filePath = ".env")
    {
        if (!File.Exists(filePath))
        {
            Console.WriteLine($"⚠️ File '{filePath}' not found.");
            return;
        }

        foreach (var line in File.ReadAllLines(filePath))
        {
            var trimmed = line.Trim();

            // Skip empty lines and comments
            if (string.IsNullOrWhiteSpace(trimmed) || trimmed.StartsWith("#"))
                continue;

            var parts = trimmed.Split('=', 2); // Only split at the first '='
            if (parts.Length == 2)
            {
                var key = parts[0].Trim();
                var value = parts[1].Trim().Trim('"'); // Remove surrounding quotes
                Environment.SetEnvironmentVariable(key, value);
            }
        }
    }
}


Installed Packages Microsoft.SemanticKernel, 1.61.0 Microsoft.SemanticKernel.Experimental.Agents, 1.18.1-alpha

In [14]:
EnvLoader.Load(); // Loads from .env

// Example: retrieve a specific environment variable
string modelId = Environment.GetEnvironmentVariable("modelId");
string endPoint = Environment.GetEnvironmentVariable("endPoint");
string apiKey = Environment.GetEnvironmentVariable("apiKey");
Console.WriteLine($"modelId: {modelId}");
Console.WriteLine($"endPoint: {endPoint}");
Console.WriteLine($"apiKey: {apiKey}");


modelId: gpt-4.1
endPoint: https://dis-azopen-ai.openai.azure.com/
apiKey: 7vddAIvhIyaYiOumOwzB2Anip91Y2JHlKjEt5y5JEAEry4nfbVQqJQQJ99BGACHYHv6XJ3w3AAABACOGaCg3


In [15]:
var builder = Kernel.CreateBuilder();
            builder.Services.AddAzureOpenAIChatCompletion(
                modelId,
                endPoint,
                apiKey
            );

            var kernel = builder.Build();

In [16]:
 #pragma warning disable SKEXP0101  
  // Track agents for clean-up
  readonly List<IAgent> _agents = new();

  IAgentThread? _agentsThread = null;
 


In [19]:
  private async Task CleanUpAsync()
  {
    Console.WriteLine("🧽 Cleaning up ...");

        if (_agentsThread != null)
        {
          Console.WriteLine("Thread going away ...");
    #pragma warning disable SKEXP0101
          await _agentsThread.DeleteAsync();
    #pragma warning restore SKEXP0101
          _agentsThread = null;
        }

    if (_agents.Any())
    {
      Console.WriteLine("Agents going away ...");
    #pragma warning disable SKEXP0101
      await Task.WhenAll(_agents.Select(agent => agent.DeleteAsync()));
    #pragma warning restore SKEXP0101
      _agents.Clear();
    }
  }


In [ ]:
async Task Execute()
{
    #pragma warning disable SKEXP0101
    // create agent in code
    var codeAgent = await new AgentBuilder()
        .WithOpenAIChatCompletion(modelId, apiKey)
        .WithInstructions("Repeat the user message in the voice of a pirate " +
                          "and then end with parrot sounds.")
        .WithName("CodeParrot")
        .WithDescription("A fun chat bot that repeats the user message in the" +
                         " voice of a pirate.")
        .BuildAsync();
    _agents.Add(codeAgent);

    // Create agent from file
    var pathToPlugin = Path.Combine(Directory.GetCurrentDirectory(), "Agents", "ParrotAgent.yaml");
    string agentDefinition = File.ReadAllText(pathToPlugin);
    var fileAgent = await new AgentBuilder()
        .WithOpenAIChatCompletion(modelId, apiKey)
        .FromTemplatePath(pathToPlugin)
        .BuildAsync();
    _agents.Add(fileAgent);
    #pragma warning restore SKEXP0101

    try
    {
        // Invoke agent plugin.
        #pragma warning disable SKEXP0101
        var response =
            await fileAgent.AsPlugin().InvokeAsync(
                "Practice makes perfect.",
                new KernelArguments { { "count", 2 } }
            );
        #pragma warning restore SKEXP0101

        // Display result.
        #pragma warning disable SKEXP0101
        Console.WriteLine(response ?? $"No response from agent: {fileAgent.Id}");
        #pragma warning restore SKEXP0101
    }
    finally
    {
        // Clean-up (storage costs $)
        await CleanUpAsync();
        #pragma warning disable SKEXP0101
        await fileAgent.DeleteAsync();
        await codeAgent.DeleteAsync();
        #pragma warning restore SKEXP0101
    }

    Console.ReadLine();
}